<a href="https://colab.research.google.com/github/farofang/chathai/blob/tf-idf/quiz1/tf-idf/quiz1_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.8-bin-hadoop2.7.tgz
!pip install findspark

--2021-09-23 17:10:16--  https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 135.181.209.10, 88.99.95.219, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235899716 (225M) [application/x-gzip]
Saving to: ‘spark-2.4.8-bin-hadoop2.7.tgz.1’

spark-2.4.8-bin-had 100%[===================>] 224.97M  11.9MB/s    in 17s     

2021-09-23 17:10:34 (13.0 MB/s) - ‘spark-2.4.8-bin-hadoop2.7.tgz.1’ saved [235899716/235899716]

spark-2.4.8-bin-hadoop2.7/
spark-2.4.8-bin-hadoop2.7/LICENSE
spark-2.4.8-bin-hadoop2.7/NOTICE
spark-2.4.8-bin-hadoop2.7/R/
spark-2.4.8-bin-hadoop2.7/R/lib/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/DESCRIPTION
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/INDEX
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-2.4.8-bin-hadoop2.7/R/lib/SparkR/Meta

In [ ]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

--2021-09-23 17:10:41--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12002039 (11M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar.1’

spark-streaming-kaf 100%[===================>]  11.45M  --.-KB/s    in 0.1s    

2021-09-23 17:10:41 (95.0 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar.1’ saved [12002039/12002039]



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random
import pyspark
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from uuid import uuid1
import time
import re
import math

kafka_topic_name = "chathai-streams-harry-tf-idf-input"
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 17:10:42


In [ ]:
sc = pyspark.SparkContext()
ssc = StreamingContext(sc,5)

kvs = KafkaUtils.createStream(ssc, kafka_bootstrap_servers, 'spark-streaming-consumer', {kafka_topic_name:1}) 
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {"metadata.broker.list": kafka_bootstrap_servers})
kvs = KafkaUtils.createDirectStream(ssc, [kafka_topic_name], {
                        'bootstrap.servers':kafka_bootstrap_servers,
                        'group.id':'test-group',
                        'auto.offset.reset':'largest'})

# set: count lines
# clines = 2      #for demo
clines = 348  #for harry book

lines = kvs.map(lambda x: x[1][1:-1].lower())
lines.pprint()

# map: doc_id, text
map1=lines.map(lambda x: (x.split('|')[0],len(x.split('|')[1].split()),x.split('|')[1]))
map1.pprint()
# map: (doc_id, word), count
map2=map1.flatMap(lambda x: [((x[0],x[1],i),1) for i in x[2].split()])
map2.pprint()

# reduce by key: (doc_id, word), total
gword = map2.reduceByKey(lambda x,y:x+y)
gword.pprint()

# tf
# map : word, (doc_id, tf)
tf = gword.map(lambda x: (x[0][2],(x[0][0],x[1]/x[0][1])))
tf.pprint()

# idf
# map: word, (doc_id, tf, 1)
map3 = gword.map(lambda x: (x[0][2],(x[0][0],x[1]/x[0][1],1)))
# map3.pprint()
# map: word, count (document)
map4 = map3.map(lambda x:(x[0],x[1][2]))
# map4.pprint()
# reduce by key: word, total (document)
gword2 = map4.reduceByKey(lambda x,y:x+y)
# gword2.pprint()
# 
idf = gword2.map(lambda x: (x[0],math.log10(clines/x[1])))
idf.pprint()

rdd=tf.join(idf)
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))
rdd.pprint()

result = rdd.filter(lambda x: x[1][-1] > 0.01)
result.pprint()

result = result.map(lambda x: (x[0],x[1][0])).reduceByKey(lambda a,b: a +', '+ b)
result.pprint()

ssc.start()
# stream will run for 50 sec
ssc.awaitTerminationOrTimeout(50)
ssc.stop()
sc.stop()

-------------------------------------------
Time: 2021-09-23 18:25:25
-------------------------------------------
1|the boy who lived mr and mrs dursley of number four privet drive were proud to say that they were perfectly normal thank you very much they were the last people you d expect to be involved in anything strange or mysterious because they just didn t hold with such nonsense mr dursley was the director of a firm called grunnings which made drills he was a big beefy man with hardly any neck although he did have a very large mustache mrs dursley was thin and blonde and had nearly twice the usual amount of neck which came in very useful as she spent so much of her time craning over garden fences spying on the neighbors the dursley s had a small son called dudley and in their opinion there was no finer boy anywhere the dursleys had everything they wanted but they also had a secret and their greatest fear was that somebody would discover it they didn t think they could bear it if 

In [ ]:
ssc.stop()
sc.stop()